In [ ]:
using DelimitedFiles,HDF5, PyCall, PyPlot, NRRD,FileIO, TiffImages,Images, ProgressMeter, Statistics,LinearAlgebra, NaNStatistics
using _Data

In [ ]:
@pyimport numpy
@pyimport matplotlib.patches as patches
@pyimport scipy.ndimage as ndimage

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_all_exp = 
[
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220407_152537", 4, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
corr_trials_all_fish = []
for which_combo = 1:length(data_all_exp)

analyzer = "chuyu"
data_combo = data_all_exp[which_combo]
experiment_filename_1 = data_combo[1]
server_1 = data_combo[2]

experimenter = data_combo[end]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")



all_files = readdir(joinpath(data_path(ds_save_cy_1), "place_cell_windows"))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)
save_file_name = candidate_filename[which_file][2]
println(save_file_name)
info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
close(file)

all_files = readdir(joinpath(data_path(ds_save_cy_1), ""))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
info_filename = joinpath(data_path(ds_save_cy_1),"", save_file_name)
file = h5open(info_filename, "r")
place_cell_index = HDF5.readmmap(file["place_cell_index"])
close(file)
corr_trials_all = fill(NaN32, size(place_map_all_1,3))

for neuron_idx in intersect(place_cell_index)
    map_1 = place_map_all_1[:,:, neuron_idx]
    map_2 = place_map_all_2[:,:, neuron_idx]
    corr_trials_all[neuron_idx] = corr_2d_original(map_1, map_2)
end
    
    append!(corr_trials_all_fish, corr_trials_all[place_cell_index])
end


    

In [ ]:
nanmedian(float32.(corr_trials_all_fish))

In [ ]:
nanstd(float32.(corr_trials_all_fish))

In [ ]:
which_combo = 1
analyzer = "chuyu"
data_combo = data_all_exp[which_combo]
experiment_filename_1 = data_combo[1]
server_1 = data_combo[2]

experimenter = data_combo[end]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


all_files = readdir(joinpath(data_path(ds_save_cy_1), ""))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
info_filename = joinpath(data_path(ds_save_cy_1),"", save_file_name)
file = h5open(info_filename, "r")
specificity = HDF5.readmmap(file["specificity"])
close(file)

    
    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)


n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end
    
    
top_cells = findall(isfinite.(specificity))[reverse(sortperm(specificity[isfinite.(specificity)]))[1:100]]
    


In [ ]:
top_cells_percent = []
for which_combo = 1:length(data_all_exp)
analyzer = "chuyu"
data_combo = data_all_exp[which_combo]
experiment_filename_1 = data_combo[1]
server_1 = data_combo[2]

experimenter = data_combo[end]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


all_files = readdir(joinpath(data_path(ds_save_cy_1), ""))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
info_filename = joinpath(data_path(ds_save_cy_1),"", save_file_name)
file = h5open(info_filename, "r")
specificity = HDF5.readmmap(file["specificity"])
close(file)

    
    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)


n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end
    
    
top_cells = findall(isfinite.(specificity))[reverse(sortperm(specificity[isfinite.(specificity)]))[1:100]]
    
append!(top_cells_percent, sum(whether_tel[top_cells])/100)
end

In [ ]:
top_cells_percent

In [ ]:
mean(top_cells_percent)

In [ ]:
std(top_cells_percent)